In [ ]:
from transformers import GenerationConfig
from peft import AutoPeftModelForCausalLM, PeftModel
from transformers import AutoModelForCausalLM
import torch
import os
import pandas as pd

In [ ]:
model_name = "PROVIDE_MODEL"

In [ ]:

train_df = pd.read_csv("PROVIDE_DS_DATA") # Provide dataset of distant supervision in prompt style


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name) 
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, load_in_8bit=False,
                                             device_map="auto",
                                             trust_remote_code=True)

model_path = "OUTPUT_DIRECTORY"

peft_model = PeftModel.from_pretrained(model, model_path, from_transformers=True, device_map="auto")

model = peft_model.merge_and_unload()

In [ ]:

def generate_output(user_input):
    prompt = formatted_prompt(user_input)

    inputs = tokenizer([prompt], return_tensors="pt")
    generation_config = GenerationConfig(penalty_alpha=0.6,do_sample = True,
      top_k=5,temperature=0.3,repetition_penalty=1.2,
      max_new_tokens=512,pad_token_id=tokenizer.eos_token_id
    )

    inputs = tokenizer(prompt, return_tensors="pt").to('cuda')

    outputs = model.generate(**inputs, generation_config=generation_config)
    result = tokenizer.decode(outputs[0], skip_special_tokens=True).split('Example Output')[1]

    return result

In [ ]:
train_df['output1'] = train_df['prompt'].apply(lambda x: generate_output(x))
train_df['output2'] = train_df['prompt'].apply(lambda x: generate_output(x))
train_df['output3'] = train_df['prompt'].apply(lambda x: generate_output(x))

In [ ]:
### Save the train_df